In [17]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('data_final.csv')

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

# Fix: Don't use the get() method with default values
# Instead, use fixed values for epochs and batch_size
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,  # Fixed value instead of best_hp.get('tuner/epochs', 50)
    batch_size=64,  # Fixed value instead of best_hp.get('tuner/batch_size', 64)
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Save model
best_model.save('best_classification_model.keras')
print("Saved classification model")

Reloading Tuner from tuner_dir/classification/tuner0.json
Epoch 1/50


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.5819 - auc: 0.5856 - loss: 0.6792 - val_accuracy: 0.5029 - val_auc: 0.5009 - val_loss: 0.7010
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6204 - auc: 0.6444 - loss: 0.6582 - val_accuracy: 0.5000 - val_auc: 0.5075 - val_loss: 0.7089
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6246 - auc: 0.6715 - loss: 0.6517 - val_accuracy: 0.5115 - val_auc: 0.5122 - val_loss: 0.7160
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6119 - auc: 0.6614 - loss: 0.6514 - val_accuracy: 0.5374 - val_auc: 0.5215 - val_loss: 0.7221
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6427 - auc: 0.7059 - loss: 0.6352 - val_accuracy: 0.5086 - val_auc: 0.5290 - val_loss: 0.7431
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6582 - auc: 0.7440 - loss: 0.6082 - val_accuracy: 0.5115 - val_auc: 0.5459 - val_loss: 0.7569
Test Loss: 0.8666, Accuracy: 0.4516, AUC: 0.4580
14/14 ━━

In [21]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('data_final.csv')

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

# Fix: Don't use the get() method with default values
# Instead, use fixed values for epochs and batch_size
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,  # Fixed value instead of best_hp.get('tuner/epochs', 50)
    batch_size=64,  # Fixed value instead of best_hp.get('tuner/batch_size', 64)
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Print model summary
print("\n=== Model Architecture ===")
best_model.summary()

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

# Feature Importance using custom approach
print("\n=== Feature Importance Analysis ===")

# Get feature names
feature_names = X.columns if hasattr(X, 'columns') else [f"Feature_{i}" for i in range(X.shape[1])]

# Analyze feature importance at the most recent time point (most relevant for prediction)
print("Analyzing feature importance at most recent time point...")
X_recent = X_test[:, -1, :]  # Most recent data point in each sequence

# Manual implementation of permutation importance
n_features = X_recent.shape[1]
n_repeats = 5
importances = np.zeros(n_features)
importances_std = np.zeros(n_features)

# Get baseline score with original data
baseline_preds = best_model.predict(X_test, verbose=0)
baseline_score = roc_auc_score(y_test, baseline_preds)
print(f"Baseline ROC AUC: {baseline_score:.4f}")

# Compute feature importance
for feature_idx in range(n_features):
    print(f"Computing importance for feature {feature_idx+1}/{n_features}: {feature_names[feature_idx] if feature_idx < len(feature_names) else 'Feature_'+str(feature_idx)}")
    
    feature_scores = []
    
    # Repeat permutation multiple times
    for _ in range(n_repeats):
        # Create a copy of the test data
        X_test_permuted = X_test.copy()
        
        # Permute the values of the feature at the most recent time step
        permuted_values = X_recent[:, feature_idx].copy()
        np.random.shuffle(permuted_values)
        X_test_permuted[:, -1, feature_idx] = permuted_values
        
        # Get predictions with permuted feature
        permuted_preds = best_model.predict(X_test_permuted, verbose=0)
        
        # Calculate score with permuted feature
        permuted_score = roc_auc_score(y_test, permuted_preds)
        
        # Calculate importance as decrease in performance
        importance = baseline_score - permuted_score
        feature_scores.append(importance)
    
    # Store mean and std of importance scores
    importances[feature_idx] = np.mean(feature_scores)
    importances_std[feature_idx] = np.std(feature_scores)



# Sort features by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("\nFeature ranking by importance:")
for i, idx in enumerate(indices):
    if i < 20:  # Print top 20 features
        feature_name = feature_names[idx] if idx < len(feature_names) else f"Feature_{idx}"
        print(f"{i+1}. {feature_name}: {importances[idx]:.4f} Â± {importances_std[idx]:.4f}")

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.title("Feature Importance (Top 15)")
top_indices = indices[:15]
plt.barh(range(len(top_indices)), importances[top_indices], color="r", yerr=importances_std[top_indices], align="center")
plt.yticks(range(len(top_indices)), [feature_names[i] if i < len(feature_names) else f"Feature_{i}" for i in top_indices])
plt.ylim([-1, len(top_indices)])
plt.xlabel("Feature Importance (Mean Decrease in AUC)")
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()
print("Saved feature importance visualization to 'feature_importance.png'")

# Save model
best_model.save('best_classification_model.keras')
print("\nSaved classification model")

Reloading Tuner from tuner_dir/classification/tuner0.json
Epoch 1/50


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.5397 - auc: 0.5571 - loss: 0.6870 - val_accuracy: 0.4770 - val_auc: 0.5177 - val_loss: 0.7002
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6126 - auc: 0.6255 - loss: 0.6627 - val_accuracy: 0.4799 - val_auc: 0.5226 - val_loss: 0.7062
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.6182 - auc: 0.6564 - loss: 0.6529 - val_accuracy: 0.5000 - val_auc: 0.5343 - val_loss: 0.7129
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6555 - auc: 0.7032 - loss: 0.6293 - val_accuracy: 0.4885 - val_auc: 0.5468 - val_loss: 0.7434
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6565 - auc: 0.7184 - loss: 0.6231 - val_accuracy: 0.5086 - val_auc: 0.5585 - val_loss: 0.7556
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6645 - auc: 0.7210 - loss: 0.6245 - val_accuracy: 0.4914 - val_auc: 0.5588 - val_loss: 0.7952

=== Model Architecture ===


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_40                │ (None, 30, 128)        │        45,568 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_41                │ (None, 30, 128)        │        74,496 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_20 (Attention)        │ (None, 128)            │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,421 (1.76 MB)

 Trainable params: 153,473 (599.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 306,948 (1.17 MB)


Test Loss: 0.9598, Accuracy: 0.4447, AUC: 0.4646
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.45      0.86      0.59       200
           1       0.43      0.09      0.15       234

    accuracy                           0.44       434
   macro avg       0.44      0.47      0.37       434
weighted avg       0.44      0.44      0.35       434


=== Confusion Matrix ===
[[172  28]
 [213  21]]

=== Feature Importance Analysis ===
Analyzing feature importance at most recent time point...
Baseline ROC AUC: 0.4649
Computing importance for feature 1/24: nonfarm_payrolls
Computing importance for feature 2/24: corporate_profits
Computing importance for feature 3/24: consumer_confidence
Computing importance for feature 4/24: permits
Computing importance for feature 5/24: unemployment_lag1
Computing importance for feature 6/24: interest_rate_roll3_std
Computing importance for feature 7/24: Gold_

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('data_final.csv')

# Binary target: up/down movement
data['Direction'] = (data['LogReturn'] > 0).astype(int)

y = data['Direction']
X = data.drop(['LogReturn', 'Direction', 'date'], axis=1, errors='ignore')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Attention layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(name='auc')]
    )
    return model

# Hyperparameter Tuning (TimeSeriesSplit CV)
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=15,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='classification'
)

tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=100,
        batch_size=64,
        callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=10), keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=5)],
        verbose=0
    )

# Retrieve and train best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

# Fix: Don't use the get() method with default values
# Instead, use fixed values for epochs and batch_size
best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,  # Fixed value instead of best_hp.get('tuner/epochs', 50)
    batch_size=64,  # Fixed value instead of best_hp.get('tuner/batch_size', 64)
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Print model summary
print("\n=== Model Architecture ===")
best_model.summary()

# Evaluate
loss, acc, auc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Detailed metrics
y_pred = (best_model.predict(X_test) > 0.5).astype(int)

# Calculate accuracy, precision, recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Feature Importance using custom approach
print("\n=== Feature Importance Analysis ===")

# Get feature names
feature_names = X.columns if hasattr(X, 'columns') else [f"Feature_{i}" for i in range(X.shape[1])]

# Analyze feature importance at the most recent time point (most relevant for prediction)
print("Analyzing feature importance at most recent time point...")
X_recent = X_test[:, -1, :]  # Most recent data point in each sequence

# Manual implementation of permutation importance
n_features = X_recent.shape[1]
n_repeats = 5
importances = np.zeros(n_features)
importances_std = np.zeros(n_features)

# Get baseline score with original data
baseline_preds = best_model.predict(X_test, verbose=0)
baseline_score = roc_auc_score(y_test, baseline_preds)
print(f"Baseline ROC AUC: {baseline_score:.4f}")

# Compute feature importance
for feature_idx in range(n_features):
    print(f"Computing importance for feature {feature_idx+1}/{n_features}: {feature_names[feature_idx] if feature_idx < len(feature_names) else 'Feature_'+str(feature_idx)}")
    
    feature_scores = []
    
    # Repeat permutation multiple times
    for _ in range(n_repeats):
        # Create a copy of the test data
        X_test_permuted = X_test.copy()
        
        # Permute the values of the feature at the most recent time step
        permuted_values = X_recent[:, feature_idx].copy()
        np.random.shuffle(permuted_values)
        X_test_permuted[:, -1, feature_idx] = permuted_values
        
        # Get predictions with permuted feature
        permuted_preds = best_model.predict(X_test_permuted, verbose=0)
        
        # Calculate score with permuted feature
        permuted_score = roc_auc_score(y_test, permuted_preds)
        
        # Calculate importance as decrease in performance
        importance = baseline_score - permuted_score
        feature_scores.append(importance)
    
    # Store mean and std of importance scores
    importances[feature_idx] = np.mean(feature_scores)
    importances_std[feature_idx] = np.std(feature_scores)

# Sort features by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("\nFeature ranking by importance:")
for i, idx in enumerate(indices):
    if i < 20:  # Print top 20 features
        feature_name = feature_names[idx] if idx < len(feature_names) else f"Feature_{idx}"
        print(f"{i+1}. {feature_name}: {importances[idx]:.4f} Â± {importances_std[idx]:.4f}")

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.title("Feature Importance (Top 15)")
top_indices = indices[:15]
plt.barh(range(len(top_indices)), importances[top_indices], color="r", yerr=importances_std[top_indices], align="center")
plt.yticks(range(len(top_indices)), [feature_names[i] if i < len(feature_names) else f"Feature_{i}" for i in top_indices])
plt.ylim([-1, len(top_indices)])
plt.xlabel("Feature Importance (Mean Decrease in AUC)")
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()
print("Saved feature importance visualization to 'feature_importance.png'")

# Save model
best_model.save('best_classification_model.keras')
print("\nSaved classification model")


2025-05-13 15:57:28.282593: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 15:57:28.444868: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 15:57:28.521672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747151848.624170    1790 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747151848.651665    1790 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747151848.842233    1790 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 138ms/step - accuracy: 0.5270 - auc: 0.5318 - loss: 0.6900 - val_accuracy: 0.5287 - val_auc: 0.4855 - val_loss: 0.7052
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.5890 - auc: 0.6259 - loss: 0.6647 - val_accuracy: 0.4713 - val_auc: 0.4926 - val_loss: 0.7147
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.6193 - auc: 0.6640 - loss: 0.6517 - val_accuracy: 0.5057 - val_auc: 0.5016 - val_loss: 0.7217
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.6240 - auc: 0.6737 - loss: 0.6448 - val_accuracy: 0.4943 - val_auc: 0.4975 - val_loss: 0.7432
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.6092 - auc: 0.6833 - loss: 0.6397 - val_accuracy: 0.5172 - val_auc: 0.5178 - val_loss: 0.7580
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.6519 - auc: 0.6867 - loss: 0.6339 - val_accuracy: 0.5000 - val_auc: 0.5120 - val_loss: 0.8122

=== Model Architecture ===


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 30, 128)        │        45,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 30, 512)        │       592,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_1 (Attention)         │ (None, 512)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 384)            │       196,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,509,061 (9.57 MB)

 Trainable params: 836,353 (3.19 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,672,708 (6.38 MB)


Test Loss: 0.9574, Accuracy: 0.4447, AUC: 0.5240
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.45      0.84      0.58       200
           1       0.44      0.10      0.17       234

    accuracy                           0.44       434
   macro avg       0.44      0.47      0.37       434
weighted avg       0.44      0.44      0.36       434


=== Confusion Matrix ===
[[169  31]
 [210  24]]

Accuracy: 0.4447
Precision: 0.4364
Recall: 0.1026

=== Feature Importance Analysis ===
Analyzing feature importance at most recent time point...
Baseline ROC AUC: 0.5232
Computing importance for feature 1/24: nonfarm_payrolls
Computing importance for feature 2/24: corporate_profits
Computing importance for feature 3/24: consumer_confidence
Computing importance for feature 4/24: permits
Computing importance for feature 5/24: unemployment_lag1
Computing importance for feature 6/24: interest_rate_rol

In [12]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('transformed_data.csv')

# Use Close as the target variable for regression
y = data['LogReturn'].values
X = data.drop(['LogReturn', 'Direction', 'date', 'Close'], axis=1, errors='ignore')

# Scale features (X)
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X)

# Scale target (y = Close)
y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y.reshape(-1, 1)).flatten()

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y_scaled, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Save original (unscaled) y_test values for final evaluation
_, y_orig = make_sequences(X_scaled, y, WINDOW)
y_test_orig = y_orig[split:]

# Attention Layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss=Huber(),
        metrics=['mae']
    )
    return model

# Hyperparameter tuning
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='regression_close'
)

tscv = TimeSeriesSplit(n_splits=3)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=50,
        batch_size=64,
        callbacks=[
            keras.callbacks.EarlyStopping('val_loss', patience=5),
            keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=3)
        ],
        verbose=0
    )

# Train final model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Predict and inverse transform
y_pred_scaled = best_model.predict(X_test).flatten()
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Evaluation
mse = mean_squared_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_orig, y_pred)
r2 = r2_score(y_test_orig, y_pred)

print(f"\nTest MSE: {mse:.6f}")
print(f"Test RMSE: {rmse:.6f}")
print(f"Test MAE: {mae:.6f}")
print(f"Test R²: {r2:.4f}")

# Save model
best_model.save('best_close_model.keras')
print("Saved Close price prediction model.")



Reloading Tuner from tuner_dir/regression_close/tuner0.json
Epoch 1/50


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 179ms/step - loss: 0.0478 - mae: 0.2083 - val_loss: 0.2391 - val_mae: 0.6039
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 166ms/step - loss: 0.0038 - mae: 0.0678 - val_loss: 0.2204 - val_mae: 0.5845
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - loss: 0.0026 - mae: 0.0558 - val_loss: 0.2024 - val_mae: 0.5632
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 164ms/step - loss: 0.0022 - mae: 0.0516 - val_loss: 0.1892 - val_mae: 0.5465
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 164ms/step - loss: 0.0017 - mae: 0.0453 - val_loss: 0.1815 - val_mae: 0.5322
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - loss: 0.0017 - mae: 0.0451 - val_loss: 0.1729 - val_mae: 0.5216
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - loss: 0.0014 - mae: 0.0415 - val_loss: 0.1707 - val_mae: 0.5176
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - loss: 0.0013 - mae: 0.0404 - val_loss: 0.1581 - val_mae: 0.4954
Epoch 9/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - loss: 0.001

In [10]:
print("Min:", y_scaled.min())
print("Max:", y_scaled.max())

Min: -1.7209867234379428
Max: 2.1450165824050687


In [8]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Huber
import kerastuner as kt

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load data
data = pd.read_csv('transformed_data.csv')

# Use LogReturn as the target variable
y = data['LogReturn'].values
X = data.drop(['LogReturn', 'Direction', 'date', 'Close'], axis=1, errors='ignore')

# Scale features (X)
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X)

# Do NOT scale target (y)
y_scaled = y  # Keep y unscaled

# Sequence creation
def make_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

WINDOW = 30
X_seq, y_seq = make_sequences(X_scaled, y_scaled, WINDOW)

# Train/test split
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Save original y_test for final evaluation
_, y_orig = make_sequences(X_scaled, y, WINDOW)
y_test_orig = y_orig[split:]

# Attention Layer
def AttentionLayer():
    class Attention(layers.Layer):
        def __init__(self, **kwargs): super().__init__(**kwargs)
        def build(self, input_shape):
            self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
        def call(self, inputs):
            att = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=[2,0]), axis=1)
            return tf.reduce_sum(inputs * tf.expand_dims(att, -1), axis=1)
    return Attention()

# Model builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hp.Int('lstm1', 64, 256, step=64), return_sequences=True), input_shape=X_train.shape[1:]))
    model.add(layers.Bidirectional(layers.GRU(hp.Int('gru1', 64, 256, step=64), return_sequences=True)))
    model.add(AttentionLayer())
    model.add(layers.Dense(hp.Int('dense', 128, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-5, 1e-2, sampling='log')),
        loss=Huber(),
        metrics=['mae']
    )
    return model

# Hyperparameter tuning
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='regression_logreturn'
)

# TimeSeries Cross Validation
tscv = TimeSeriesSplit(n_splits=3)
for train_idx, val_idx in tscv.split(X_train):
    tuner.search(
        X_train[train_idx], y_train[train_idx],
        validation_data=(X_train[val_idx], y_train[val_idx]),
        epochs=50,
        batch_size=64,
        callbacks=[
            keras.callbacks.EarlyStopping('val_loss', patience=5),
            keras.callbacks.ReduceLROnPlateau('val_loss', factor=0.5, patience=3)
        ],
        verbose=0
    )

# Train final model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)],
    verbose=1
)

# Predict (no inverse transform needed)
y_pred = best_model.predict(X_test).flatten()

# Evaluation
mse = mean_squared_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_orig, y_pred)
r2 = r2_score(y_test_orig, y_pred)

print(f"\nTest MSE: {mse:.6f}")
print(f"Test RMSE: {rmse:.6f}")
print(f"Test MAE: {mae:.6f}")
print(f"Test R²: {r2:.4f}")

# Save model
best_model.save('best_logreturn_model.keras')
print("Saved LogReturn prediction model.")


Reloading Tuner from tuner_dir/regression_logreturn/tuner0.json


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 195ms/step - loss: 0.3737 - mae: 0.6900 - val_loss: 0.2250 - val_mae: 0.4754
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - loss: 0.0157 - mae: 0.1403 - val_loss: 0.3874 - val_mae: 0.6491
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - loss: 0.0121 - mae: 0.1234 - val_loss: 0.3169 - val_mae: 0.5418
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 170ms/step - loss: 0.0107 - mae: 0.1168 - val_loss: 0.2789 - val_mae: 0.4971
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - loss: 0.0082 - mae: 0.1001 - val_loss: 0.2389 - val_mae: 0.4500
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - loss: 0.0079 - mae: 0.0985 - val_loss: 0.2469 - val_mae: 0.4762
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step

Test MSE: 10.615928
Test RMSE: 3.258209
Test MAE: 2.892967
Test R²: -630.4324
Saved LogReturn prediction model.
